In [1]:
import csv
import librosa 
import numpy as np
import librosa.display
import matplotlib.pyplot as plt
import pandas as pd
import random
import math

from librosa.core import load


data = [] # список для извлеченных тренировочных данных
data_augmentation = [] # список для искуственно созданных данных
data_test = []

data_label = [] # список для меотк извлеченных тренировочных данных
data_augmentation_label = [] # список для меток искуственно созданных данных
data_test_label = []

# переменные, нужные для аугментации данных
#yTrain = 'beach bus cafe/restaurant car city_center forest_path grocery_store home library metro_station office park residential_area train tram'.split()
yTrain = '1 2 3 4 5 6 7 8 9 10 11 12 13 14 15'.split()
avallable_Spectrogram = []
count = 0

#data_train_dir = '/Users/sedoy/Desktop/Work Files/Программирование/Нейросеть/mini_dataset/data_train/'
#data_test_dir = '/Users/sedoy/Desktop/Work Files/Программирование/Нейросеть/mini_dataset/data_for_test/'

data_train_dir = '/Users/sedoy/Desktop/Work Files/Программирование/Нейросеть/data for train/'
data_test_dir = '/Users/sedoy/Desktop/Work Files/Программирование/Нейросеть/data for test/'

lambd = 0.5 
idx = 0 # параметр для выбора рандомного значения индекса при смешивании
n = 0
score = 0 # параметр для вычисления количества элементов базе data 
add_line = [] # список для добавления искусственных данных
new_data = []


def transfrom_data(x, way): # функция извлечения данных 
    
    eps = np.finfo(np.float32).eps # для логорифмирования
    massive = [] # пустой массив для загрузкки каналов
    massive_for_mfc = [] # пустой массив для загрузкки mfc
    data10 = [] # пустой массив для загрузкки 10 секундный отрезков
    n = 0 # локальная переменная для цикла
    
    way_2 = textik(x, way) # задаем путь через функцию textic
    y, sr = librosa.load(way_2, sr=None, mono=False) 
    b=np.row_stack((np.sum(y,0), np.diff(y, axis=0))) # средняя кодировка
    b = np.ascontiguousarray(b) 
    
    # разделяем 1-й и 2-й каналы на секундные отрезки
    b_patch1 = librosa.util.frame(b[0], frame_length=44100, hop_length=22050, axis = 0)
    b_patch2 = librosa.util.frame(b[1], frame_length=44100, hop_length=22050, axis = 0)
    
    while n != 19:
        # объеденяем два канала секундного отрезка n в один массив
        massive.insert(0, b_patch1[n])
        massive.insert(1, b_patch2[n])
        massive = np.array(massive)
        
        # извлекаем mfc каждого из каналов
        S1 = librosa.feature.melspectrogram(y=massive[0,:], sr=44100, n_fft = 2048, hop_length = 1024, win_length = 2048, n_mels=128, center = False)
        S2 = librosa.feature.melspectrogram(y=massive[1,:], sr=44100, n_fft = 2048, hop_length = 1024, win_length = 2048, n_mels=128, center = False)
        
        # логорифмирование
        S1 = np.log10(S1 + eps)
        S2 = np.log10(S2 + eps)
        
        # загружаем извлеченные mfc в финальный массив
        massive_for_mfc.insert(0, S1)
        massive_for_mfc.insert(1, S2)
        massive_for_mfc = np.array(massive_for_mfc)
        massive_for_mfc = np.moveaxis(massive_for_mfc, [0,1,2], [2,0,1])
        
        data10.append(massive_for_mfc) # добавляем полученный 3-х мерный массив в возвращаемый массив
        massive_for_mfc.tolist()
        massive_for_mfc = []
        massive = []
        n +=1

    return data10

def textik(x, dir_way): # функция для определения директории 
    way = dir_way + str(x)
    return way

def augmentation(x_line,idx, mix): # функция-формула для расширения данных
    mfc = mix[0]
    #x = x_line[0] # функция работает для старого варианта - не использовать
    x = x_line
    new_data = lambd*x + (1-lambd)*mfc
    return new_data

# преобразование текстовой метки в численное значение для последущей векторизации
def label_categorial(label):
    if label == 'beach':
        x = 0
    elif label == 'bus':
        x = 1
    elif label == 'cafe/restaurant':
        x = 2
    elif label == 'car':
        x = 3
    elif label == 'city_center':
        x = 4
    elif label == 'forest_path':
        x = 5
    elif label == 'grocery_store':
        x = 6
    elif label == 'home':
        x = 7
    elif label == 'library':
        x = 8
    elif label == 'metro_station':
        x = 9
    elif label == 'office':
        x = 10
    elif label == 'park':
        x = 11
    elif label == 'residential_area':
        x = 12
    elif label == 'train':
        x = 13
    elif label == 'tram':
        x = 14
    else:
        x = label
        
    return x


# обработка тестовых данных
with open('/Users/sedoy/Desktop/Work Files/Программирование/Нейросеть/data for test/data_for_test.csv', 'r', encoding = 'utf-8-sig') as f:
    csv_reader = csv.reader(f,delimiter=',')
    for line in csv_reader:
        n2 = 0
        mark = label_categorial(line[1]) # метка 
        mark = str(mark)
        data_with10 = transfrom_data(line[0], data_test_dir)
        while n2 != 19:
            data_test.append(data_with10[n2])
            data_test_label.append(mark)
            n2 +=1



        
    

data_test = np.array(data_test)
data_test_label = np.array(data_test_label)


    
with open('data_test.npy', 'wb') as f:
    np.save(f, data_test, allow_pickle = False , fix_imports = False)
    
with open('data_test_label.npy', 'wb') as f:
    np.save(f, data_test_label, allow_pickle = False , fix_imports = False)

    
print('Количество фрагментов тестовых данных:', len(data_test))


Количество фрагментов тестовых данных: 29450


In [2]:
import csv
import librosa 
import numpy as np
import librosa.display
import matplotlib.pyplot as plt
import pandas as pd
import random
import math

from librosa.core import load


data = [] # список для извлеченных тренировочных данных
data_augmentation = [] # список для искуственно созданных данных
data_test = []

data_label = [] # список для меотк извлеченных тренировочных данных
data_augmentation_label = [] # список для меток искуственно созданных данных
data_test_label = []

# переменные, нужные для аугментации данных
#yTrain = 'beach bus cafe/restaurant car city_center forest_path grocery_store home library metro_station office park residential_area train tram'.split()
yTrain = '1 2 3 4 5 6 7 8 9 10 11 12 13 14 15'.split()
avallable_Spectrogram = []
count = 0

#data_train_dir = '/Users/sedoy/Desktop/Work Files/Программирование/Нейросеть/mini_dataset/data_train/'
#data_test_dir = '/Users/sedoy/Desktop/Work Files/Программирование/Нейросеть/mini_dataset/data_for_test/'

data_train_dir = '/Users/sedoy/Desktop/Work Files/Программирование/Нейросеть/data for train/'
data_test_dir = '/Users/sedoy/Desktop/Work Files/Программирование/Нейросеть/data for test/'

lambd = 0.5 
idx = 0 # параметр для выбора рандомного значения индекса при смешивании
n = 0
score = 0 # параметр для вычисления количества элементов базе data 
add_line = [] # список для добавления искусственных данных
new_data = []


def transfrom_data(x, way): # функция извлечения данных 
    
    eps = np.finfo(np.float32).eps # для логорифмирования
    massive = [] # пустой массив для загрузкки каналов
    massive_for_mfc = [] # пустой массив для загрузкки mfc
    data10 = [] # пустой массив для загрузкки 10 секундный отрезков
    n = 0 # локальная переменная для цикла
    
    way_2 = textik(x, way) # задаем путь через функцию textic
    y, sr = librosa.load(way_2, sr=None, mono=False) 
    b=np.row_stack((np.sum(y,0), np.diff(y, axis=0))) # средняя кодировка
    b = np.ascontiguousarray(b) 
    
    # разделяем 1-й и 2-й каналы на секундные отрезки
    b_patch1 = librosa.util.frame(b[0], frame_length=44100, hop_length=22050, axis = 0)
    b_patch2 = librosa.util.frame(b[1], frame_length=44100, hop_length=22050, axis = 0)
    
    while n != 19:
        # объеденяем два канала секундного отрезка n в один массив
        massive.insert(0, b_patch1[n])
        massive.insert(1, b_patch2[n])
        massive = np.array(massive)
        
        # извлекаем mfc каждого из каналов
        S1 = librosa.feature.melspectrogram(y=massive[0,:], sr=44100, n_fft = 2048, hop_length = 1024, win_length = 2048, n_mels=128, center = False)
        S2 = librosa.feature.melspectrogram(y=massive[1,:], sr=44100, n_fft = 2048, hop_length = 1024, win_length = 2048, n_mels=128, center = False)
        
        # логорифмирование
        S1 = np.log10(S1 + eps)
        S2 = np.log10(S2 + eps)
        
        # загружаем извлеченные mfc в финальный массив
        massive_for_mfc.insert(0, S1)
        massive_for_mfc.insert(1, S2)
        massive_for_mfc = np.array(massive_for_mfc)
        massive_for_mfc = np.moveaxis(massive_for_mfc, [0,1,2], [2,0,1])
        
        data10.append(massive_for_mfc) # добавляем полученный 3-х мерный массив в возвращаемый массив
        massive_for_mfc.tolist()
        massive_for_mfc = []
        massive = []
        n +=1

    return data10

def textik(x, dir_way): # функция для определения директории 
    way = dir_way + str(x)
    return way

def augmentation(x_line,idx, mix): # функция-формула для расширения данных
    mfc = mix
    mfc = mix[0] # функция работает для старого варианта - не использовать
    #x = x_line[0] # функция работает для старого варианта - не использовать
    x = x_line
    new_data = lambd*x + (1-lambd)*mfc
    return new_data

# преобразование текстовой метки в численное значение для последущей векторизации
def label_categorial(label):
    if label == 'beach':
        x = 0
    elif label == 'bus':
        x = 1
    elif label == 'cafe/restaurant':
        x = 2
    elif label == 'car':
        x = 3
    elif label == 'city_center':
        x = 4
    elif label == 'forest_path':
        x = 5
    elif label == 'grocery_store':
        x = 6
    elif label == 'home':
        x = 7
    elif label == 'library':
        x = 8
    elif label == 'metro_station':
        x = 9
    elif label == 'office':
        x = 10
    elif label == 'park':
        x = 11
    elif label == 'residential_area':
        x = 12
    elif label == 'train':
        x = 13
    elif label == 'tram':
        x = 14
    else:
        x = label
        
    return x




with open('/Users/sedoy/Desktop/Work Files/Программирование/Нейросеть/data for train/data_train_way .csv', 'r', encoding = 'utf-8-sig') as f:
    csv_reader = csv.reader(f,delimiter=',')
    for line in csv_reader:
        n2 = 0
        mark = label_categorial(line[1]) # метка 
        mark = str(mark)        
        data_with10 = transfrom_data(line[0], data_train_dir)
        while n2 != 19:
            data.append(data_with10[n2])
            data_label.append(mark)
            n2 +=1
            score +=1
    
    
print('Количество фрагментов тренировочных данных:', len(data))
print()
print('data.shape', np.array(data).shape)
print('data_label.shape', np.array(data_label).shape)


spec = np.array(data)
xTrainExtra=np.array(data)
yTrainExtra=np.array(data_label)
lamda=.5
yT=np.array(data_label)


for q in range (0,spec.shape[0]):
    a=yT[q]
    availableSpectrograms=np.argwhere(yT != a)
    numAvailableSpectrograms= len(availableSpectrograms)
    idx=random.randint(0,numAvailableSpectrograms-1)
    xTrainExtra[q,:,:,:]=lamda*spec[q,:,:,:]+(1-lamda)*spec[availableSpectrograms[idx,0],:,:,:]
    
    if random.random()>lamda:
        yTrainExtra[q]=yT[availableSpectrograms[idx,0]]
        
xTrainFull=np.concatenate((spec, xTrainExtra)) 
yTrainFull=np.concatenate((yT, yTrainExtra))


with open('data_aug.npy', 'wb') as f:
    np.save(f, xTrainFull, allow_pickle = False , fix_imports = False)
    
with open('data_label_aug.npy', 'wb') as f:
    np.save(f, yTrainFull, allow_pickle = False , fix_imports = False)
    

    
print('Количество фрагментов аугментироованных данных:', len(xTrainExtra))
print('Итоговое количество фрагментов тренировочных данных:', len(xTrainFull))

Количество фрагментов тренировочных данных: 88920

data.shape (88920, 128, 42, 2)
data_label.shape (88920,)
Количество фрагментов аугментироованных данных: 88920
Итоговое количество фрагментов тренировочных данных: 177840
